In [9]:
import gym #Importa a biblioteca Gym, que é usada para criar e manipular ambientes de simulação de aprendizado por reforço.
import random #Importa a biblioteca Random, que fornece funções para gerar números aleatórios.

random.seed(1234) #Define a semente para o gerador de números aleatórios.

# Cria uma instância do ambiente Taxi-v3 usando a biblioteca Gym
# render_mode='ansi' é usado para renderizar o ambiente como texto que pode ser impresso no console
streets = gym.make("Taxi-v3", render_mode='ansi').env

# Reinicializa o ambiente para o estado inicial, pronto para começar um novo episódio
streets.reset()

# Renderiza o ambiente como texto e imprime a representação do estado inicial no console
print ("\n" + streets.render())


+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+




In [10]:
initial_state =  streets.encode(2, 3, 2, 0)

streets.s = initial_state

streets.render()

'+---------+\n|R: | : :\x1bG\x1b|\n| : | : : |\n| : : : : |\n| | : | : |\n|Y| :\x1b \x1b|\x1bB\x1b: |\n+---------+\n\n'

In [11]:
# Acessa as transições possíveis no ambiente a partir do estado inicial 
# Isso retorna um dicionário onde as chaves são ações e os valores são listas de transições possíveis
# Cada transição é uma tupla (probabilidade, próximo_estado, recompensa, feito)
streets.P[initial_state]

{0: [(1.0, 368, -1, False)],
 1: [(1.0, 168, -1, False)],
 2: [(1.0, 288, -1, False)],
 3: [(1.0, 248, -1, False)],
 4: [(1.0, 268, -10, False)],
 5: [(1.0, 268, -10, False)]}

In [18]:
import numpy as np  

# Cria uma Q-table inicializada com zeros, com dimensões baseadas no número de estados e ações possíveis no ambiente
q_table = np.zeros([streets.observation_space.n, streets.action_space.n])

# Define a taxa de aprendizado, o fator de desconto, a taxa de exploração e o número de épocas (episódios) de treinamento
learning_rate = 0.1  # Taxa de aprendizado - define o quanto as novas informações substituem as antigas
discount_factor = 0.6  # Fator de desconto - define a importância das recompensas futuras
exploration = 0.1  # Taxa de exploração - define a probabilidade de escolher uma ação aleatória (explorar) versus a melhor ação conhecida (explorar)
epochs = 10000  # Número de épocas - define quantas vezes o agente vai jogar o jogo para aprender

# Loop principal para rodar múltiplos episódios de treinamento
for taxi_run in range(epochs):
    # Reinicializa o ambiente para o estado inicial e captura o estado inicial e informações adicionais
    state, info = streets.reset()
    done = False  # Inicializa a variável 'done' para verificar se o episódio terminou
    
    # Loop interno para cada passo dentro do episódio
    while not done:
        # Gera um valor aleatório entre 0 e 1 para decidir entre exploração e exploração
        random_value = random.uniform(0, 1)
        if random_value < exploration:
            # Escolhe uma ação aleatória do espaço de ações do ambiente (exploração)
            action = streets.action_space.sample()
        else:
            # Escolhe a melhor ação com base nos valores Q da Q-table para o estado atual (explorar)
            action = np.argmax(q_table[state])

        # Executa a ação escolhida e observa o próximo estado, a recompensa, se o episódio terminou (done), se foi truncado (truncated) e informações adicionais (info)
        next_state, reward, done, truncated, info = streets.step(action)

        # Armazena o valor Q atual para o par (estado, ação)
        prev_q = q_table[state, action]
        # Encontra o melhor valor Q para o próximo estado
        next_max_q = np.max(q_table[next_state])
        # Calcula o novo valor Q usando a fórmula de atualização do Q-learning
        new_q = (1 - learning_rate) * prev_q + learning_rate * (reward + discount_factor * next_max_q)
        # Atualiza a Q-table com o novo valor Q
        q_table[state, action] = new_q

        # Atualiza o estado atual para o próximo estado
        state = next_state
        # Verifica se o episódio terminou ou foi truncado e atualiza a variável 'done'
        done = done or truncated


In [19]:
 # Acessa a linha da tabela Q (q_table) que corresponde ao estado inicial (initial_state), retornando os valores Q para todas as ações possíveis neste estado.

q_table[initial_state]

# array([-2.40148903, -2.39484223, -2.39833537, -2.3639511 , -8.97576456, -8.04553863]) 
# -2.40148903: Valor Q para a ação 0 (Sul)
# -2.39484223: Valor Q para a ação 1 (Norte)
# -2.39833537: Valor Q para a ação 2 (Leste)
# -2.3639511: Valor Q para a ação 3 (Oeste) 
# -8.97576456: Valor Q para a ação 4 (Pegar Passageiro)
# -8.04553863: Valor Q para a ação 5 (Deixar Passageiro)
# Valores negativos indicam penalidades ou custos associados às ações no estado inicial. Ações com valores menos negativos são consideradas melhores pelo algoritmo.

array([-2.41219737, -2.40876978, -2.39719389, -2.3639511 , -8.93846373,
       -6.30556416])

In [20]:
from IPython.display import clear_output  # Importa a função clear_output da biblioteca IPython para limpar a saída da célula
from time import sleep  # Importa a função sleep da biblioteca time para pausar a execução do código

# Loop principal que executa 10 viagens no ambiente Taxi-v3
for tripnum in range(1, 11):
    state, info = streets.reset()  # Reinicializa o ambiente e captura o estado inicial e informações adicionais
    
    # Verificação para garantir que `state` é um inteiro
    if not isinstance(state, int):
        raise ValueError(f"O estado deve ser um inteiro, mas é {type(state)}: {state}")
    
    done = False  # Inicializa a variável `done` como False para indicar que o episódio não terminou
    
    # Loop interno que continua até que `done` seja True, ou seja, até que o episódio termine
    while not done:
        action = np.argmax(q_table[state])  # Escolhe a ação com o maior valor Q no estado atual usando a Q-table
        # Executa a ação escolhida no ambiente e captura o próximo estado, recompensa, se o episódio terminou,
        # se foi truncado e informações adicionais
        next_state, reward, done, truncated, info = streets.step(action)
        
        clear_output(wait=True)  # Limpa a saída da célula atual no notebook Jupyter
        print("Trip number " + str(tripnum))  # Imprime o número da viagem atual
        print(streets.render())  # Renderiza o ambiente atual como texto e imprime no console
        
        sleep(0.5)  # Pausa a execução do código por 0.5 segundos para tornar a visualização do ambiente mais lenta
        
        state = next_state  # Atualiza o estado atual para o próximo estado
    
    sleep(2)  # Pausa a execução do código por 2 segundos antes de iniciar a próxima viagem


Trip number 10
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)



Desafio!:

Modifique o bloco acima para acompanhar o total de passos de tempo e use isso como uma métrica para avaliar a eficácia do nosso sistema de Q-learning. Você pode querer aumentar o número de viagens simuladas e remover as chamadas sleep() para permitir que o sistema seja executado com mais amostras.

Agora, experimente ajustar os hiperparâmetros. Até que ponto o número de épocas pode ser reduzido antes que nosso modelo comece a sofrer? Você consegue encontrar melhores taxas de aprendizado, fatores de desconto ou fatores de exploração para tornar o treinamento mais eficiente? A taxa de exploração vs. exploração, em particular, é interessante para experimentar.